<a href="https://colab.research.google.com/github/cristianscu99-VIU/03MIAR_Algoritmos-de-Optimizacion/blob/main/SEMINARIO/Feedback_Proyecto_AOP_Problema_2_Cristian_de_la_Santa_Cruz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: **Cristian de la Santa Cruz Urbano**   <br>
Url: https://github.com/cristianscu99-VIU/AOP-ACT-1/blob/main/SEMINARIO/Feedback_Proyecto_AOP_Problema_2_Cristian_de_la_Santa_Cruz.ipynb<br>
Problema:
> 1. Sesiones de doblaje <br>
>2. **Organizar los horarios de partidos de La Liga**<br>
>3. Combinar cifras y operaciones

**Descripción del problema:**

- Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.

- Los horarios disponibles se conocen a priori y son los siguientes:


In [ ]:
# @title
  Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                 'D20' : 1, 'L20' : 0.4}
  listado_horarios = list(Ponderacion.keys())
  listado_horarios

['V20', 'S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20', 'L20']

- En primer lugar se clasifican los equipos en tres categorías según el numero de seguidores( que tiene relación directa con la audiencia). Hay 3 equiposen la categoría A,11 equiposde categoría By 6 equiposde categoría C.

- Se conoce estadísticamente la audiencia que genera cada partido según los equipos que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos)

In [ ]:
# @title
  import pandas as pd
  audiencia_base = {
      'A' : {'A' : 2},
      'B' : {'A' : 1.3, 'B' : 0.9},
      'C' : {'A' : 1, 'B' : 0.75, 'C' : 0.47}
  }
  df = pd.DataFrame(audiencia_base)
  display(df)

,A,B,C
A,2.0,1.3,1.00
B,NaN,0.9,0.75
C,NaN,NaN,0.47


- Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce según los coeficientes de la siguiente tabla
- Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes

In [ ]:
# @title
Ponderacion = {'Viernes' : {'20h' : 0.4},
             'Sábado' : {'12h' : 0.55, '16h' : 0.7, '18h' : 0.8, '20h' : 1},
             'Domingo' : {'12h' : 0.45, '16h' : 0.75, '18h' : 0.85, '20h' : 1},
             'Lunes' : {'20h' : 0.4}
             }
Ponderacion = pd.DataFrame(Ponderacion).sort_index(ascending=True)
display(Ponderacion)

,Viernes,Sábado,Domingo,Lunes
12h,NaN,0.55,0.45,NaN
16h,NaN,0.70,0.75,NaN
18h,NaN,0.80,0.85,NaN
20h,0.4,1.00,1.00,0.4


- Es posible la coincidencia de horarios pero en este caso la audiencia de cada partido se verá afectada y se estima que se reduce en porcentaje según la siguiente tabla dependiendo del número de coincidencias:

In [ ]:
# @title
Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}

df_Coincidencias = pd.DataFrame(Coincidencias)
df_Coincidencias.columns.name='Coincidencias'
display(df_Coincidencias)

Coincidencias,-%
0,0.00
1,0.25
2,0.45
3,0.60
4,0.70
5,0.75
6,0.78
7,0.80
8,0.80


- Los cálculos asociados a una jornada de ejemplo se realizan según se muestra en la siguiente tabla:

In [ ]:
# @title
import random
import numpy as np
import pandas as pd

dict_equipos = {'Celta' : 'B', 'Real Madrid' : 'A', 'Valencia' : 'B', 'R.Sociedad' : 'A', 'Mallorca' : 'C', 'Eibar' : 'C',
           'Athletic' : 'B', 'Barcelona' : 'A', 'Leganés' : 'C', 'Osasuna' : 'C', 'Villareal' : 'B', 'Granada' : 'C',
           'Alavés' : 'B', 'Levante' : 'B', 'Espayol' : 'B', 'Sevilla' : 'B', 'Betis' : 'B', 'Valladolid' : 'C', 'Atlético' : 'B',
           'Getafe' : 'B'
           }

def Correcion_por_coincidencia(lista_horario_partido): # devuelve una lista con las diferentes correciones en base a la coincidencia
  Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}
  lista_correciones = []

  for hora in lista_horario_partido:
    contador = lista_horario_partido.count(hora) - 1
    lista_correciones.append(Coincidencias['-%'][contador])
  return lista_correciones

def generar_partidos (dict_equipos):
  "Función que devuelve los partidos que se van a jugar"
  lista_partidos = [] # lista que guarda los distintos partidos de una jornada
  lista_categorias = [] # lista que guarda las distintas categorias de los partidos de una jornada
  lista_bases = [] # lista con las bases de audiencia de cada partido en funcion de la categoría
  lista_equipos = list(dict_equipos.keys()) #lista con todos los equipos
  lista_equipos_disponibles = lista_equipos[:]

  lista_partidos = [
      'Celta - Real Madrid',
      'Valencia - R.Sociedad',
      'Mallorca - Eibar',
      'Athletic - Barcelona',
      'Leganés - Osasuna',
      'Villareal - Granada',
      'Alavés - Levante',
      'Espayol - Sevilla',
      'Betis - Valladolid',
      'Atlético - Getafe',
  ]

  lista_categorias = [
      'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B', 'B-C', 'B-B'
  ]

  lista_bases = [
      1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75, 0.9
  ]


  Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                 'D20' : 1, 'L20' : 0.4}

  listado_horarios = list(Ponderacion.keys())

  lista_horario_partido = [
      'V20', 'S12', 'S16', 'S18', 'S20', 'D16', 'D16', 'D18', 'D20', 'L20'
  ]

  lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]

  return lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones

lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones = generar_partidos(dict_equipos)
lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)

dict_jornada = {
    'Partido' : lista_partidos,
    'Categorías' : lista_categorias,
    'Horario' : lista_horario_partido,
    'Base(Mill.)' : lista_bases,
    'Ponderación' : lista_ponderaciones,
    'Base*Ponderación' : lista_base_ponderacion,
    'Correción Coincidencia' : lista_correciones_corregida,
}

df_jornada = pd.DataFrame(dict_jornada)
display(df_jornada)
Total = np.sum(lista_correciones_corregida)
print('\n Total: ',Total)

,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,V20,1.30,0.40,0.5200,0.520000
1,Valencia - R.Sociedad,B-A,S12,1.30,0.55,0.7150,0.715000
2,Mallorca - Eibar,C-C,S16,0.47,0.70,0.3290,0.329000
3,Athletic - Barcelona,B-A,S18,1.30,0.80,1.0400,1.040000
4,Leganés - Osasuna,C-C,S20,0.47,1.00,0.4700,0.470000
5,Villareal - Granada,B-C,D16,0.75,0.75,0.5625,0.421875
6,Alavés - Levante,B-B,D16,0.90,0.75,0.6750,0.506250
7,Espayol - Sevilla,B-B,D18,0.90,0.85,0.7650,0.765000
8,Betis - Valladolid,B-C,D20,0.75,1.00,0.7500,0.750000
9,Atlético - Getafe,B-B,L20,0.90,0.40,0.3600,0.360000



 Total:  5.877125


# Preguntas

**(*) ¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>**



**¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.**

Respuesta
Teniendo en cuenta que el problema se acota para una única jornada

- a) Cómo la cantidad de partidos es P = 10 y la cantidad de horarios distintos es H = 10, tenemos que H^P = 10^10 = 1*10^10, es decir, 10 mil millones de combinaciones posibles.

- b) Sabiendo que las restricciones son que obligatoriamente debe haber al menos un partido el Viernes y otro partido el Lunes y se corresponden con el primer y último horario respectivamente. Tenemos que la cantidad de soluciones a calcular sería alrededor de 4 mil 100 millones de combinaciones posibles.

In [ ]:
H = 10 # Cantidad de horarios distintos. H = len(listado_horarios)
P = 10 # Cantidad de partidos distintos en una jornada. P = len(lista_partidos)
print(f'a) Combinatoria sin restricciones: {H**P}')
print(f'\nb) Combinatoria con restricciones:{H**P} - {(H-1)**P} - {(H-1)**P} + {(H-2)**P} = {H**P - (H-1)**P - (H-1)**P + (H-2)**P}')

a) Combinatoria sin restricciones: 10000000000

b) Combinatoria con restricciones:10000000000 - 3486784401 - 3486784401 + 1073741824 = 4100173022


Explicación:

Para calcular el número de posibles soluciones es necesario calcular a través de combinatoria la cantidad de situaciones diferentes a evaluar.
- **Sin restrincciones:**
Suponiendo que tengamos P partidos en 1 jornada, y H horarios diferentes posibles para poder jugar cada partido. Tendríamos que ir evaluando el resultado de la audiencia de esa jornada para cada partido. Cada partido tiene H horarios diferentes en los que poder jugar.
Supongamos el caso en que P = 4 y H = 3

In [ ]:
# @title
from itertools import product

P = 4
H = 3

# Generar todas las combinaciones de P números del 0 a H-1
matriz = list(product(range(H), repeat=P))

print(f'La cantidad de combinaciones diferentes de horarios entre partidos es {len(matriz)}'
      f'\nque es lo mismo que la combinatoria H^(P), es decir {H}^{P} = {H**P}. \n\n Combinatoria:')
matriz

# Index - PARTIDO
# Value - HORARIO (0 es el primer horario)

La cantidad de combinaciones diferentes de horarios entre partidos es 81
que es lo mismo que la combinatoria H^(P), es decir 3^4 = 81. 

 Combinatoria:


[(0, 0, 0, 0),
 (0, 0, 0, 1),
 (0, 0, 0, 2),
 (0, 0, 1, 0),
 (0, 0, 1, 1),
 (0, 0, 1, 2),
 (0, 0, 2, 0),
 (0, 0, 2, 1),
 (0, 0, 2, 2),
 (0, 1, 0, 0),
 (0, 1, 0, 1),
 (0, 1, 0, 2),
 (0, 1, 1, 0),
 (0, 1, 1, 1),
 (0, 1, 1, 2),
 (0, 1, 2, 0),
 (0, 1, 2, 1),
 (0, 1, 2, 2),
 (0, 2, 0, 0),
 (0, 2, 0, 1),
 (0, 2, 0, 2),
 (0, 2, 1, 0),
 (0, 2, 1, 1),
 (0, 2, 1, 2),
 (0, 2, 2, 0),
 (0, 2, 2, 1),
 (0, 2, 2, 2),
 (1, 0, 0, 0),
 (1, 0, 0, 1),
 (1, 0, 0, 2),
 (1, 0, 1, 0),
 (1, 0, 1, 1),
 (1, 0, 1, 2),
 (1, 0, 2, 0),
 (1, 0, 2, 1),
 (1, 0, 2, 2),
 (1, 1, 0, 0),
 (1, 1, 0, 1),
 (1, 1, 0, 2),
 (1, 1, 1, 0),
 (1, 1, 1, 1),
 (1, 1, 1, 2),
 (1, 1, 2, 0),
 (1, 1, 2, 1),
 (1, 1, 2, 2),
 (1, 2, 0, 0),
 (1, 2, 0, 1),
 (1, 2, 0, 2),
 (1, 2, 1, 0),
 (1, 2, 1, 1),
 (1, 2, 1, 2),
 (1, 2, 2, 0),
 (1, 2, 2, 1),
 (1, 2, 2, 2),
 (2, 0, 0, 0),
 (2, 0, 0, 1),
 (2, 0, 0, 2),
 (2, 0, 1, 0),
 (2, 0, 1, 1),
 (2, 0, 1, 2),
 (2, 0, 2, 0),
 (2, 0, 2, 1),
 (2, 0, 2, 2),
 (2, 1, 0, 0),
 (2, 1, 0, 1),
 (2, 1, 0, 2),
 (2, 1, 1,

- **Con restricciones:** Teniendo en cuenta todas las restricciones, tendríamos que analizar todas aquellas posibles soluciones que contengan como partida al menos 1 partido en el primer horario (Viernes) y un partido en el último horario (Lunes). Suponemos P=4 y H=3.

In [ ]:
from itertools import product

P = 4
H = 3

# Generar todas las combinaciones de P números del 0 a H-1
matriz = list(product(range(H), repeat=P))
matriz_restringida = [tuple for tuple in matriz if (0 in tuple and H-1 in tuple)]

print(f'La cantidad de combinaciones diferentes de horarios entre partidos con restricciones es {len(matriz_restringida)}.'
      f'\nQue es lo mismo que la combinatoria H^(P) - A - B + AintersecciónB,'
      f'\nsiendo A el conjunto formado por las filas que no contienen el primer horario'
      f'\ny B el conjunto formado por las filas que no contienen el último horario, es decir:'
      f'\nH = {[i for i in range(H)]}, A = {H-1}^P (A es subconjunto de H que no tiene 0) y de la misma forma,'
      f'\nB = {H-1}^P (B es subconjunto de H que no tiene {H-1}) y'
      f'\nAintersecciónB = {H-2}^P (AintersecciónB es subconjunto de H que no tiene 0 ni {H-1})'
      f'\nPor tanto: H^(P) - A - B + AintersecciónB = {H**P}-{(H-1)**P}-{(H-1)**P}+{(H-2)**P} = {H**P-2*(H-1)**P+(H-2)**P} \n\n Combinatoria:')

matriz_restringida

# Index - PARTIDO
# Value - HORARIO (0 es el primer horario)

La cantidad de combinaciones diferentes de horarios entre partidos con restricciones es 50.
Que es lo mismo que la combinatoria H^(P) - A - B + AintersecciónB,
siendo A el conjunto formado por las filas que no contienen el primer horario
y B el conjunto formado por las filas que no contienen el último horario, es decir:
H = [0, 1, 2], A = 2^P (A es subconjunto de H que no tiene 0) y de la misma forma,
B = 2^P (B es subconjunto de H que no tiene 2) y
AintersecciónB = 1^P (AintersecciónB es subconjunto de H que no tiene 0 ni 2)
Por tanto: H^(P) - A - B + AintersecciónB = 81-16-16+1 = 50 

 Combinatoria:


[(0, 0, 0, 2),
 (0, 0, 1, 2),
 (0, 0, 2, 0),
 (0, 0, 2, 1),
 (0, 0, 2, 2),
 (0, 1, 0, 2),
 (0, 1, 1, 2),
 (0, 1, 2, 0),
 (0, 1, 2, 1),
 (0, 1, 2, 2),
 (0, 2, 0, 0),
 (0, 2, 0, 1),
 (0, 2, 0, 2),
 (0, 2, 1, 0),
 (0, 2, 1, 1),
 (0, 2, 1, 2),
 (0, 2, 2, 0),
 (0, 2, 2, 1),
 (0, 2, 2, 2),
 (1, 0, 0, 2),
 (1, 0, 1, 2),
 (1, 0, 2, 0),
 (1, 0, 2, 1),
 (1, 0, 2, 2),
 (1, 1, 0, 2),
 (1, 1, 2, 0),
 (1, 2, 0, 0),
 (1, 2, 0, 1),
 (1, 2, 0, 2),
 (1, 2, 1, 0),
 (1, 2, 2, 0),
 (2, 0, 0, 0),
 (2, 0, 0, 1),
 (2, 0, 0, 2),
 (2, 0, 1, 0),
 (2, 0, 1, 1),
 (2, 0, 1, 2),
 (2, 0, 2, 0),
 (2, 0, 2, 1),
 (2, 0, 2, 2),
 (2, 1, 0, 0),
 (2, 1, 0, 1),
 (2, 1, 0, 2),
 (2, 1, 1, 0),
 (2, 1, 2, 0),
 (2, 2, 0, 0),
 (2, 2, 0, 1),
 (2, 2, 0, 2),
 (2, 2, 1, 0),
 (2, 2, 2, 0)]

**Modelo para el espacio de soluciones**<br>
**(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguméntalo)**

Respuesta

En un primer escenario he propuesto la siguiente estructuras de datos:
Diccionarios : dict_equipos, Coincidencias
Listas : lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones, lista_base_ponderacion,lista_correciones, lista_correciones_corregida
DataFrames : df_jornada

Modificar: ahora son np.arrays la gran mayoría y la combinatoria no se guarda porque sería excesivo. Usar otra codificación no sería eficiente por la arquitectura del pocesador.

**Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?**

**(*)¿Es un problema de maximización o minimización?**

Respuesta

La función objetivo es el cálculo de la audiencia en base a la organización horaria de los partidos (lista_horario_partido).

Se trata de un problema de maximización de audiencia.

**Diseña un algoritmo para resolver el problema por fuerza bruta**

In [51]:
H = 3
P = 10


def gen_h(H,P):
  horarios = [0 for x in range(P)]
  for i in range(H**P):
    print(i)
    horarios[0] = i//10**(P-1)
    resto = i%10**(P-1)
    horarios[1] = resto//10**(P-2)
    resto = i%10**(P-2)
    horarios[2] = resto//10**(P-3)
    resto = i%10**(P-3)
    horarios[3] = resto//10**(P-4)
    resto = i%10**(P-4)
    horarios[4] = resto//10**(P-5)
    resto = i%10**(P-5)
    horarios[5] = resto//10**(P-6)
    resto = i%10**(P-6)
    horarios[6] = resto//10**(P-7)
    resto = i%10**(P-7)
    horarios[7] = resto//10**(P-8)
    resto = i%10**(P-8)
    horarios[8] = resto//10**(P-9)
    resto = i%10**(P-9)
    horarios[9] = resto
    yield tuple(horarios)


horarios = gen_h(H,P)
for i, fila in enumerate(horarios):
  lista = []
  lista.append(fila)
  print(lista)





# for i in range(H**P):
#   print(i)
#   horarios[0] = i//10**P
#   resto = i%10**P
#   horarios[1] = resto//10**(P-1)
#   resto = i%10**(P-1)
#   horarios[2] = resto//10**(P-2)
#   resto = i%10**(P-2)
#   horarios[3] = resto//10**(P-3)
#   resto = i%10**(P-3)
#   horarios[4] = resto//10**(P-4)
#   resto = i%10**(P-4)
#   horarios[5] = resto//10**(P-5)
#   resto = i%10**(P-5)
#   horarios[6] = resto//10**(P-6)
#   resto = i%10**(P-6)
#   horarios[7] = resto//10**(P-7)
#   resto = i%10**(P-7)
#   horarios[8] = resto//10**(P-8)
#   resto = i%10**(P-8)
#   horarios[9] = resto//10**(P-9)
#   resto = i%10**(P-9)
#   print(horarios)


Se han truncado las últimas 5000 líneas del flujo de salida.
56549
[(0, 0, 0, 0, 0, 5, 6, 5, 4, 9)]
56550
[(0, 0, 0, 0, 0, 5, 6, 5, 5, 0)]
56551
[(0, 0, 0, 0, 0, 5, 6, 5, 5, 1)]
56552
[(0, 0, 0, 0, 0, 5, 6, 5, 5, 2)]
56553
[(0, 0, 0, 0, 0, 5, 6, 5, 5, 3)]
56554
[(0, 0, 0, 0, 0, 5, 6, 5, 5, 4)]
56555
[(0, 0, 0, 0, 0, 5, 6, 5, 5, 5)]
56556
[(0, 0, 0, 0, 0, 5, 6, 5, 5, 6)]
56557
[(0, 0, 0, 0, 0, 5, 6, 5, 5, 7)]
56558
[(0, 0, 0, 0, 0, 5, 6, 5, 5, 8)]
56559
[(0, 0, 0, 0, 0, 5, 6, 5, 5, 9)]
56560
[(0, 0, 0, 0, 0, 5, 6, 5, 6, 0)]
56561
[(0, 0, 0, 0, 0, 5, 6, 5, 6, 1)]
56562
[(0, 0, 0, 0, 0, 5, 6, 5, 6, 2)]
56563
[(0, 0, 0, 0, 0, 5, 6, 5, 6, 3)]
56564
[(0, 0, 0, 0, 0, 5, 6, 5, 6, 4)]
56565
[(0, 0, 0, 0, 0, 5, 6, 5, 6, 5)]
56566
[(0, 0, 0, 0, 0, 5, 6, 5, 6, 6)]
56567
[(0, 0, 0, 0, 0, 5, 6, 5, 6, 7)]
56568
[(0, 0, 0, 0, 0, 5, 6, 5, 6, 8)]
56569
[(0, 0, 0, 0, 0, 5, 6, 5, 6, 9)]
56570
[(0, 0, 0, 0, 0, 5, 6, 5, 7, 0)]
56571
[(0, 0, 0, 0, 0, 5, 6, 5, 7, 1)]
56572
[(0, 0, 0, 0, 0, 5, 6, 5, 7, 2)]
565

In [20]:
def problema_fuerza_bruta():
  import numpy as np
  import pandas as pd
  from itertools import product
  import time
  from collections import Counter

  # Función que devuelve una lista con las diferentes correciones en base a la coincidencia
  def Correcion_por_coincidencia(lista_horario_partido):
      #Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]} cogemos directamente la lista para no llamar al diccionario
      correcciones = [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]
      contador = Counter(lista_horario_partido)

      # Aplicar la corrección correspondiente a cada valor
      lista_correciones = [correcciones[min(contador[hora] - 1, len(correcciones) - 1)] for hora in lista_horario_partido]

      return np.array(lista_correciones)

  # Función que genera dataframe
  def generar_dataframe(lista_partidos: list, lista_categorias: list, lista_horario_partido:list, lista_bases: list, lista_ponderaciones:list, lista_base_ponderacion:list, lista_correciones_corregida:list ):

    dict_jornada = {
        'Partido' : lista_partidos,
        'Categorías' : lista_categorias,
        'Horario' : lista_horario_partido,
        'Base(Mill.)' : lista_bases,
        'Ponderación' : lista_ponderaciones,
        'Base*Ponderación' : lista_base_ponderacion,
        'Correción Coincidencia' : lista_correciones_corregida,
    }

    df_jornada = pd.DataFrame(dict_jornada)
    return df_jornada

  # Función que genera combinación con restrincciones
  def combinaciones_validas(P, H):
      for comb in product(range(H), repeat=P):
          if 0 in comb and (H - 1) in comb:
              yield comb


  # Condiciones iniciales para un problema fijo (P = PARTIDOS, H = HORARIOS)

  #############################################
  # PROBLEMA P = 4 H = 3
  #############################################

  # lista_partidos = [
  #     'Celta - Real Madrid',
  #     'Valencia - R.Sociedad',
  #     'Mallorca - Eibar',
  #     'Athletic - Barcelona',
  # ]

  # lista_categorias = ['B-A', 'B-A', 'C-C', 'B-A']

  # lista_bases = [1.3, 1.3, 0.47, 1.3]

  # Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7}

  #############################################
  # PROBLEMA P = 7 H = 7
  #############################################
  # lista_partidos = [
  #       'Celta - Real Madrid',
  #       'Valencia - R.Sociedad',
  #       'Mallorca - Eibar',
  #       'Athletic - Barcelona',
  #       'Leganés - Osasuna',
  #       'Villareal - Granada',
  #       'Alavés - Levante',
  #   ]

  # lista_categorias = [
  #       'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B'
  #   ]

  # lista_bases = [
  #       1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9
  #   ]


  # Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
  #                  'S20' : 1, 'D12' : 0.45, 'D16' : 0.75}

  #############################################
    # PROBLEMA P = 8 H = 8
  #############################################


  lista_partidos = [
        'Celta - Real Madrid',
        'Valencia - R.Sociedad',
        'Mallorca - Eibar',
        'Athletic - Barcelona',
        'Leganés - Osasuna',
        'Villareal - Granada',
        'Alavés - Levante',
        'Espayol - Sevilla',
    ]

  lista_categorias = [
        'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B'
    ]

  lista_bases = [
        1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9
    ]


  Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                   'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85}


  #############################################
  # PROBLEMA P = 9 H = 10
  #############################################

  # lista_partidos = [
  #   'Celta - Real Madrid',
  #   'Valencia - R.Sociedad',
  #   'Mallorca - Eibar',
  #   'Athletic - Barcelona',
  #   'Leganés - Osasuna',
  #   'Villareal - Granada',
  #   'Alavés - Levante',
  #   'Espayol - Sevilla',
  #   'Betis - Valladolid',
  #   ]

  # lista_categorias = [
  #     'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B', 'B-C'
  # ]

  # lista_bases = [
  #     1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75
  # ]


  # Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
  #                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
  #                 'D20' : 1, 'L20' : 0.4}

  ##############################################
  #############################################
    # PROBLEMA P = 10 H = 10
  #############################################

  # lista_partidos = [
  #   'Celta - Real Madrid',
  #   'Valencia - R.Sociedad',
  #   'Mallorca - Eibar',
  #   'Athletic - Barcelona',
  #   'Leganés - Osasuna',
  #   'Villareal - Granada',
  #   'Alavés - Levante',
  #   'Espayol - Sevilla',
  #   'Betis - Valladolid',
  #   'Atlético - Getafe',
  #   ]

  # lista_categorias = [
  #     'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B', 'B-C', 'B-B'
  # ]

  # lista_bases = [
  #     1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75, 0.9
  # ]


  # Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
  #                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
  #                 'D20' : 1, 'L20' : 0.4}

  ##############################################

  Total_max = 0
  horario_max_audiencia = []

  listado_horarios = list(Ponderacion.keys())

  P = len(lista_partidos)
  H = len(listado_horarios)
  combinaciones_totales = (H**P - (H-1)**P - (H-1)**P + (H-2)**P)

  # Generador de combinaciones teniendo en cuentra restrincciones
  gen = combinaciones_validas(P, H)

  start_time = time.time()  # Inicia temporizador
  # Cálculo de la audiencia para cada combinación posible
  for i, fila in enumerate(gen):
    fila_array = np.array(fila)
    lista_horario_partido = np.array(listado_horarios)[fila_array] # lista de listas con todos los horarios posibles para la jornada
    lista_ponderaciones = np.array([Ponderacion[h] for h in lista_horario_partido])
    lista_base_ponderacion = np.array(lista_bases)*lista_ponderaciones
    lista_correciones = Correcion_por_coincidencia(lista_horario_partido)
    lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)
    Total = np.sum(lista_correciones_corregida)

    if Total > Total_max:
      Total_max = Total
      #horario_max_audiencia = [] # vaciamos la lista
      horario_max_audiencia = [lista_horario_partido.tolist()]
      print(f'\nCombinación número: {i}, Máxima Audiencia encontrada: {Total_max:.2f}')
      end_time = time.time()  # Finaliza temporizador
      tiempo_transcurrido = int(end_time - start_time)
      horas_transcurridas = int(tiempo_transcurrido//3600)
      minutos_transcurridos = int((tiempo_transcurrido %3600)//60)
      segundos_transcurridos = int(tiempo_transcurrido-horas_transcurridas*3600)%60
      print(f"Tiempo total de ejecución: {horas_transcurridas} horas, {minutos_transcurridos} minutos y {segundos_transcurridos} segundos")

      if i>1:
        tiempo_estimado_total = (combinaciones_totales*(end_time - start_time)/i)
        horas = int(tiempo_estimado_total//3600)
        minutos = int((tiempo_estimado_total %3600)//60)
        segundos = int(tiempo_estimado_total-horas*3600)%60
        print(f"Tiempo total estimado: {horas} horas, {minutos} minutos y {segundos} segundos")

    elif Total == Total_max:
      horario_max_audiencia.append(lista_horario_partido.tolist())

  # ¿1 Solución o varias?
  if len(horario_max_audiencia) == 1:
    print(f'\nSolución: El horario con máxima audiencia es: {horario_max_audiencia} y obtiene una audiencia de {Total_max} \n')
  elif len(horario_max_audiencia) > 1:
    print(f'\nSolución: Los horarios con máxima audiencia son: {len(horario_max_audiencia)} y se obtiene una audiencia de {Total_max} en cualquiera de estas combinaciones \n')

  # Mostramos el DataFrame para los horarios que producen una máxima audiencia en una jornada
  for lista_horario_partido in horario_max_audiencia:
    lista_ponderaciones = np.array([Ponderacion[h] for h in lista_horario_partido])
    lista_base_ponderacion = np.array(lista_bases)*lista_ponderaciones
    lista_correciones = Correcion_por_coincidencia(lista_horario_partido)
    lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)
    df_jornada_max_audiencia = generar_dataframe(lista_partidos, lista_categorias, lista_horario_partido, lista_bases, lista_ponderaciones, lista_base_ponderacion, lista_correciones_corregida)
    display(df_jornada_max_audiencia)
    Total = np.sum(lista_correciones_corregida)
    print(f'La audiencia total esperada es: {Total} M. \n')

In [19]:
problema_fuerza_bruta()

"""
Nota: Pruebe a comentar y descomentar otros datos iniciales,
para el problema original en Colab no se podría calcular (45h) pero en mi Pc podría calcularlo en 17h.
"""

#############################################
    # PROBLEMA P = 7 H = 7 (10s)
#############################################

#############################################
    # PROBLEMA P = 8 H = 8 (4min)
#############################################

#############################################
    # PROBLEMA P = 9 H = 10 (3h 45min)
#############################################

#############################################
    # PROBLEMA P = 10 H = 10 (44h 30min)
#############################################


Combinación número: 0, Máxima Audiencia encontrada: 1.34
Tiempo total de ejecución: 0 horas, 0 minutos y 0 segundos

Combinación número: 1, Máxima Audiencia encontrada: 1.82
Tiempo total de ejecución: 0 horas, 0 minutos y 0 segundos

Combinación número: 2, Máxima Audiencia encontrada: 1.95
Tiempo total de ejecución: 0 horas, 0 minutos y 0 segundos
Tiempo total estimado: 0 horas, 21 minutos y 30 segundos

Combinación número: 3, Máxima Audiencia encontrada: 2.04
Tiempo total de ejecución: 0 horas, 0 minutos y 0 segundos
Tiempo total estimado: 0 horas, 16 minutos y 32 segundos

Combinación número: 4, Máxima Audiencia encontrada: 2.22
Tiempo total de ejecución: 0 horas, 0 minutos y 0 segundos
Tiempo total estimado: 0 horas, 13 minutos y 59 segundos

Combinación número: 17, Máxima Audiencia encontrada: 2.39
Tiempo total de ejecución: 0 horas, 0 minutos y 0 segundos
Tiempo total estimado: 0 horas, 6 minutos y 19 segundos

Combinación número: 18, Máxima Audiencia encontrada: 2.48
Tiempo tota

,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,S18,1.30,0.80,1.0400,1.0400
1,Valencia - R.Sociedad,B-A,S20,1.30,1.00,1.3000,1.3000
2,Mallorca - Eibar,C-C,V20,0.47,0.40,0.1880,0.1880
3,Athletic - Barcelona,B-A,D18,1.30,0.85,1.1050,1.1050
4,Leganés - Osasuna,C-C,D12,0.47,0.45,0.2115,0.2115
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,S16,0.90,0.70,0.6300,0.6300
7,Espayol - Sevilla,B-B,D16,0.90,0.75,0.6750,0.6750


La audiencia total esperada es: 5.562 M. 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,S18,1.30,0.80,1.0400,1.0400
1,Valencia - R.Sociedad,B-A,S20,1.30,1.00,1.3000,1.3000
2,Mallorca - Eibar,C-C,V20,0.47,0.40,0.1880,0.1880
3,Athletic - Barcelona,B-A,D18,1.30,0.85,1.1050,1.1050
4,Leganés - Osasuna,C-C,D12,0.47,0.45,0.2115,0.2115
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,D16,0.90,0.75,0.6750,0.6750
7,Espayol - Sevilla,B-B,S16,0.90,0.70,0.6300,0.6300


La audiencia total esperada es: 5.562 M. 



...

,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,D18,1.30,0.85,1.1050,1.1050
1,Valencia - R.Sociedad,B-A,S20,1.30,1.00,1.3000,1.3000
2,Mallorca - Eibar,C-C,D12,0.47,0.45,0.2115,0.2115
3,Athletic - Barcelona,B-A,S18,1.30,0.80,1.0400,1.0400
4,Leganés - Osasuna,C-C,V20,0.47,0.40,0.1880,0.1880
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,S16,0.90,0.70,0.6300,0.6300
7,Espayol - Sevilla,B-B,D16,0.90,0.75,0.6750,0.6750


La audiencia total esperada es: 5.562 M. 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,D18,1.30,0.85,1.1050,1.1050
1,Valencia - R.Sociedad,B-A,S20,1.30,1.00,1.3000,1.3000
2,Mallorca - Eibar,C-C,D12,0.47,0.45,0.2115,0.2115
3,Athletic - Barcelona,B-A,S18,1.30,0.80,1.0400,1.0400
4,Leganés - Osasuna,C-C,V20,0.47,0.40,0.1880,0.1880
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,D16,0.90,0.75,0.6750,0.6750
7,Espayol - Sevilla,B-B,S16,0.90,0.70,0.6300,0.6300


La audiencia total esperada es: 5.562 M. 



'\nNota: Pruebe a comentar y descomentar otros datos iniciales,\npara el problema original en Colab no se podría calcular (45h) pero en mi Pc podría calcularlo en 17h.\n'

**Calcula la complejidad del algoritmo por fuerza bruta**

O(2^n)

**Explicación** =

**(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta**

**Backtraking con poda** Reduce la cantidad de ramas a calcular en base a un criterio de poda, cuando la rama es inviable, el algoritmo deberá regresar al padre con mejores características descartando esa rama

In [ ]:
import sys
import math

H=10
P=10

t = tuple(range(10))
size = sys.getsizeof(t) + sum(sys.getsizeof(i) for i in t)

print(size*(H**P-2*(H-1)**P+(H-2)**P)/(2**30)) # Fuerza bruta GB
factorial = math.factorial(H)

print(size*factorial/(2**30)) # Con poda almacenar combinaciones GB realmente esto si lo podría hacer pero ¿es bueno? e incluso pesará menos si hago la Poda 2 explicada abajo

1527.4334780871868
1.3518333435058594


Aquí si necesito que me ayude y me guíe. (Abra este texto que será más legible)

Como es inabordable tener lista con todas las combinaciones posibles, en fuerza bruta utilizaba un generador y calculaba por combinación generada.

Ahora, el árbol de expansión no sé si lo voy a poder ir guardando en una estructura de datos. Esta es la idea que se me ha ocurrido:

Padre = [0,_,_,_,_,_,_,_,_,_,9]
  Max_audiencia_posible = Audiencia [P1[0]+P10[9]] + Audiencia_Base [P2:P8]
  Min_audiencia_posible = Audiencia [P1[0]+P10[9]]
    (que realmente podría llegar a ser peor si sus hijos le penalizan por coincidencias, pero debo de tomar una cota para no resolver el problema en esas combinatorias)

Hijos = [0,a,_,_,_,_,_,_,_,_,9] con a = {0 to 9} e ir guardando Max y Min de audiencia posibles tal como le explico arriba.

Hijos_2 = [0,a,b,_,_,_,_,_,_,_,9] con a, b {0 to 9}

Hijos_3 = [0,a,b,c,_,_,_,_,_,_,9] con a, b, c {0 to 9}

Hijos_4 = [0,a,b,c,d,_,_,_,_,_,9] con a, b, c, d {0 to 9}

1a Poda
Entonces, dentro de los hijos si puedo podar aquellas ramas que impliquen que cualquiera de sus hijos utilice un horario repetido en sus padres, pues por coincidencia el mínimo puede ser menor que el de sus padres y seguro que es menor que sus hermanos con una inclusión de horario diferente a los que les preceden y la cota superior se va a ver mucho más mermada, con lo cual es imposible que sean un óptimo y se deben de podar. (Esto creo que lo meteré como if condicionales antes del yield para filtrar que no haya repetidos y esa creo que será la poda reduciendo a 10! combinaciones)

2a Poda (más dificil de implementar)
Por otra parte la comparación se debe de producir entre diferentes ramas, en el momento en el que el mínimo de una rama sobrepasa al máximo de otra, esta última rama debe ser podada, pues el máximo valor posible va a ser encontrado en otra rama. (Esto creo que lo implementaré como un diccionario)

Ejemplo: [0,1,2,3,4,5,_,_,_,_,9] Max(6 + 3,5)  Min(6) -> (9.5 , 6)
         [1,2,3,4,5,0,6,_,_,_,9] Max(3,5 + 2,5)  Min(3,5) -> (6 , 3.5)
         Podar([1,2,3,4,5,0,6,_,_,_,9]) y Back([0,1,2,3,4,5,_,_,_,_,9])

Necesito saber como gestionar eso, es verdad que le tengo que dar una pensada pero si me guía me ayudará mucho, gracias.



In [ ]:
# Experimentos míos para la poda 2

def combinaciones_validas(P, H):
  matriz = []
  for comb in product(range(H), repeat=P):
      # if 0 in comb and (H - 1) in comb:
      #   print(comb)
          # yield comb
      if comb[0] == 0 and comb[len(comb)-1] == H-1:
        matriz.append(comb)
  return matriz
matriz_padre = combinaciones_validas(7, 7)

# coger todas las combinaciones que tiene (0,_,2)


comun = (0, 0, 0, 0, 0)
matriz_hijos = (0, 0, 0, 0, 0)

filtered_data = [item for item in matriz_padre if item[:len(matriz_hijos)] == matriz_hijos]
print(filtered_data)

[(0, 0, 0, 0, 0, 0, 6), (0, 0, 0, 0, 0, 1, 6), (0, 0, 0, 0, 0, 2, 6), (0, 0, 0, 0, 0, 3, 6), (0, 0, 0, 0, 0, 4, 6), (0, 0, 0, 0, 0, 5, 6), (0, 0, 0, 0, 0, 6, 6)]


In [ ]:
from itertools import permutations

def combinaciones_validas(P, H): # generador para fuerza bruta, todas las combinaciones
  matriz = []
  for comb in product(range(H), repeat=P):
      if 0 in comb and (H - 1) in comb:
        print(comb)
          yield comb


def generar_tuplas_sin_repeticion(P, H):  # generador para poda, todas las combinaciones sin repeticón de elementos
    """Genera tuplas de longitud P con valores únicos entre 0 y H-1."""
    if P > H:
        return  # No es posible generar permutaciones de longitud P sin repetir si P > H
    for comb in permutations(range(H), P):
      if 0 in comb and (H - 1) in comb:
        print(comb)
generar_tuplas_sin_repeticion(3, 7)

(0, 1, 6)
(0, 2, 6)
(0, 3, 6)
(0, 4, 6)
(0, 5, 6)
(0, 6, 1)
(0, 6, 2)
(0, 6, 3)
(0, 6, 4)
(0, 6, 5)
(1, 0, 6)
(1, 6, 0)
(2, 0, 6)
(2, 6, 0)
(3, 0, 6)
(3, 6, 0)
(4, 0, 6)
(4, 6, 0)
(5, 0, 6)
(5, 6, 0)
(6, 0, 1)
(6, 0, 2)
(6, 0, 3)
(6, 0, 4)
(6, 0, 5)
(6, 1, 0)
(6, 2, 0)
(6, 3, 0)
(6, 4, 0)
(6, 5, 0)


In [ ]:
def problema_poda():
  import numpy as np
  import pandas as pd
  from itertools import product
  import time
  from collections import Counter
  import math

  # Función que devuelve una lista con las diferentes correciones en base a la coincidencia
  def Correcion_por_coincidencia(lista_horario_partido):
      #Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]} cogemos directamente la lista para no llamar al diccionario
      correcciones = [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]
      contador = Counter(lista_horario_partido)

      # Aplicar la corrección correspondiente a cada valor
      lista_correciones = [correcciones[min(contador[hora] - 1, len(correcciones) - 1)] for hora in lista_horario_partido]

      return np.array(lista_correciones)

  # Función que genera dataframe
  def generar_dataframe(lista_partidos: list, lista_categorias: list, lista_horario_partido:list, lista_bases: list, lista_ponderaciones:list, lista_base_ponderacion:list, lista_correciones_corregida:list ):

    dict_jornada = {
        'Partido' : lista_partidos,
        'Categorías' : lista_categorias,
        'Horario' : lista_horario_partido,
        'Base(Mill.)' : lista_bases,
        'Ponderación' : lista_ponderaciones,
        'Base*Ponderación' : lista_base_ponderacion,
        'Correción Coincidencia' : lista_correciones_corregida,
    }

    df_jornada = pd.DataFrame(dict_jornada)
    return df_jornada

  # Función que genera combinación con restrincciones
  def generar_tuplas_sin_repeticion(P, H):
      """Genera tuplas de longitud P con valores únicos entre 0 y H-1."""
      if P > H:
          return  # No es posible generar permutaciones de longitud P sin repetir si P > H
      for comb in permutations(range(H), P):
        if 0 in comb and (H - 1) in comb:
          yield comb


  # Condiciones iniciales para un problema fijo (P = PARTIDOS, H = HORARIOS)

  #############################################
  # PROBLEMA P = 4 H = 3
  #############################################

  # lista_partidos = [
  #     'Celta - Real Madrid',
  #     'Valencia - R.Sociedad',
  #     'Mallorca - Eibar',
  #     'Athletic - Barcelona',
  # ]

  # lista_categorias = ['B-A', 'B-A', 'C-C', 'B-A']

  # lista_bases = [1.3, 1.3, 0.47, 1.3]

  # Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7}

  #############################################
  # PROBLEMA P = 7 H = 7
  #############################################
  # lista_partidos = [
  #       'Celta - Real Madrid',
  #       'Valencia - R.Sociedad',
  #       'Mallorca - Eibar',
  #       'Athletic - Barcelona',
  #       'Leganés - Osasuna',
  #       'Villareal - Granada',
  #       'Alavés - Levante',
  #   ]

  # lista_categorias = [
  #       'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B'
  #   ]

  # lista_bases = [
  #       1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9
  #   ]


  # Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
  #                  'S20' : 1, 'D12' : 0.45, 'D16' : 0.75}

  #############################################
    # PROBLEMA P = 8 H = 8
  #############################################


  # lista_partidos = [
  #       'Celta - Real Madrid',
  #       'Valencia - R.Sociedad',
  #       'Mallorca - Eibar',
  #       'Athletic - Barcelona',
  #       'Leganés - Osasuna',
  #       'Villareal - Granada',
  #       'Alavés - Levante',
  #       'Espayol - Sevilla',
  #   ]

  # lista_categorias = [
  #       'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B'
  #   ]

  # lista_bases = [
  #       1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9
  #   ]


  # Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
  #                  'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85}


  #############################################
  # PROBLEMA P = 9 H = 10
  #############################################

  # lista_partidos = [
  #   'Celta - Real Madrid',
  #   'Valencia - R.Sociedad',
  #   'Mallorca - Eibar',
  #   'Athletic - Barcelona',
  #   'Leganés - Osasuna',
  #   'Villareal - Granada',
  #   'Alavés - Levante',
  #   'Espayol - Sevilla',
  #   'Betis - Valladolid',
  #   ]

  # lista_categorias = [
  #     'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B', 'B-C'
  # ]

  # lista_bases = [
  #     1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75
  # ]


  # Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
  #                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
  #                 'D20' : 1, 'L20' : 0.4}

  ##############################################
  #############################################
    # PROBLEMA P = 10 H = 10
  #############################################

  lista_partidos = [
    'Celta - Real Madrid',
    'Valencia - R.Sociedad',
    'Mallorca - Eibar',
    'Athletic - Barcelona',
    'Leganés - Osasuna',
    'Villareal - Granada',
    'Alavés - Levante',
    'Espayol - Sevilla',
    'Betis - Valladolid',
    'Atlético - Getafe',
    ]

  lista_categorias = [
      'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B', 'B-C', 'B-B'
  ]

  lista_bases = [
      1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75, 0.9
  ]


  Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                  'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                  'D20' : 1, 'L20' : 0.4}

  ##############################################

  Total_max = 0
  horario_max_audiencia = []

  listado_horarios = list(Ponderacion.keys())

  P = len(lista_partidos)
  H = len(listado_horarios)
  combinaciones_totales = math.factorial(H)

  # Generador de combinaciones teniendo en cuentra restrincciones
  gen = generar_tuplas_sin_repeticion(H, P)

  start_time = time.time()  # Inicia temporizador
  # Cálculo de la audiencia para cada combinación posible
  for i, fila in enumerate(gen):
    fila_array = np.array(fila)
    lista_horario_partido = np.array(listado_horarios)[fila_array] # lista de listas con todos los horarios posibles para la jornada
    lista_ponderaciones = np.array([Ponderacion[h] for h in lista_horario_partido])
    lista_base_ponderacion = np.array(lista_bases)*lista_ponderaciones
    lista_correciones = Correcion_por_coincidencia(lista_horario_partido)
    lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)
    Total = np.sum(lista_correciones_corregida)

    if Total > Total_max:
      Total_max = Total
      #horario_max_audiencia = [] # vaciamos la lista
      horario_max_audiencia = [lista_horario_partido.tolist()]
      print(f'\nCombinación número: {i}, Máxima Audiencia encontrada: {Total_max:.2f}')
      end_time = time.time()  # Finaliza temporizador
      print(f"Tiempo total de ejecución: {int(end_time - start_time)} segundos")

      if i>1:
        tiempo_estimado_total = (combinaciones_totales*(end_time - start_time)/i)
        horas = int(tiempo_estimado_total//3600)
        minutos = int((tiempo_estimado_total %3600)//60)
        segundos = int(tiempo_estimado_total-horas*3600)%60
        print(f"Tiempo total estimado: {horas} horas, {minutos} minutos y {segundos} segundos")

    elif Total == Total_max:
      horario_max_audiencia.append(lista_horario_partido.tolist())

  # ¿1 Solución o varias?
  if len(horario_max_audiencia) == 1:
    print(f'\nSolución: El horario con máxima audiencia es: {horario_max_audiencia} y obtiene una audiencia de {Total_max} \n')
  elif len(horario_max_audiencia) > 1:
    print(f'\nSolución: Los horarios con máxima audiencia son: {len(horario_max_audiencia)} y se obtiene una audiencia de {Total_max} en cualquiera de estas combinaciones \n')

  # Mostramos el DataFrame para los horarios que producen una máxima audiencia en una jornada
  for lista_horario_partido in horario_max_audiencia:
    lista_ponderaciones = np.array([Ponderacion[h] for h in lista_horario_partido])
    lista_base_ponderacion = np.array(lista_bases)*lista_ponderaciones
    lista_correciones = Correcion_por_coincidencia(lista_horario_partido)
    lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)
    df_jornada_max_audiencia = generar_dataframe(lista_partidos, lista_categorias, lista_horario_partido, lista_bases, lista_ponderaciones, lista_base_ponderacion, lista_correciones_corregida)
    display(df_jornada_max_audiencia)
    Total = np.sum(lista_correciones_corregida)
    print(f'La audiencia total esperada es: {Total} M. \n')

In [ ]:
problema_poda()


Combinación número: 0, Máxima Audiencia encontrada: 5.96
Tiempo total de ejecución: 0 segundos

Combinación número: 1, Máxima Audiencia encontrada: 6.05
Tiempo total de ejecución: 0 segundos

Combinación número: 104, Máxima Audiencia encontrada: 6.05
Tiempo total de ejecución: 0 segundos
Tiempo total estimado: 0 horas, 5 minutos y 11 segundos

Combinación número: 120, Máxima Audiencia encontrada: 6.12
Tiempo total de ejecución: 0 segundos
Tiempo total estimado: 0 horas, 5 minutos y 9 segundos

Combinación número: 121, Máxima Audiencia encontrada: 6.21
Tiempo total de ejecución: 0 segundos
Tiempo total estimado: 0 horas, 5 minutos y 10 segundos

Combinación número: 145, Máxima Audiencia encontrada: 6.24
Tiempo total de ejecución: 0 segundos
Tiempo total estimado: 0 horas, 6 minutos y 22 segundos

Combinación número: 626, Máxima Audiencia encontrada: 6.26
Tiempo total de ejecución: 0 segundos
Tiempo total estimado: 0 horas, 4 minutos y 56 segundos

Combinación número: 841, Máxima Audien

,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,S20,1.30,1.00,1.3000,1.3000
1,Valencia - R.Sociedad,B-A,D18,1.30,0.85,1.1050,1.1050
2,Mallorca - Eibar,C-C,V20,0.47,0.40,0.1880,0.1880
3,Athletic - Barcelona,B-A,D20,1.30,1.00,1.3000,1.3000
4,Leganés - Osasuna,C-C,L20,0.47,0.40,0.1880,0.1880
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,S16,0.90,0.70,0.6300,0.6300
7,Espayol - Sevilla,B-B,S18,0.90,0.80,0.7200,0.7200
8,Betis - Valladolid,B-C,D12,0.75,0.45,0.3375,0.3375
9,Atlético - Getafe,B-B,D16,0.90,0.75,0.6750,0.6750


La audiencia total esperada es: 6.856000000000001 M. 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,S20,1.30,1.00,1.3000,1.3000
1,Valencia - R.Sociedad,B-A,D18,1.30,0.85,1.1050,1.1050
2,Mallorca - Eibar,C-C,V20,0.47,0.40,0.1880,0.1880
3,Athletic - Barcelona,B-A,D20,1.30,1.00,1.3000,1.3000
4,Leganés - Osasuna,C-C,L20,0.47,0.40,0.1880,0.1880
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,S16,0.90,0.70,0.6300,0.6300
7,Espayol - Sevilla,B-B,D16,0.90,0.75,0.6750,0.6750
8,Betis - Valladolid,B-C,D12,0.75,0.45,0.3375,0.3375
9,Atlético - Getafe,B-B,S18,0.90,0.80,0.7200,0.7200


La audiencia total esperada es: 6.856000000000001 M. 



...

La audiencia total esperada es: 6.856000000000001 M. 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,D20,1.30,1.00,1.3000,1.3000
1,Valencia - R.Sociedad,B-A,D18,1.30,0.85,1.1050,1.1050
2,Mallorca - Eibar,C-C,L20,0.47,0.40,0.1880,0.1880
3,Athletic - Barcelona,B-A,S20,1.30,1.00,1.3000,1.3000
4,Leganés - Osasuna,C-C,V20,0.47,0.40,0.1880,0.1880
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,D16,0.90,0.75,0.6750,0.6750
7,Espayol - Sevilla,B-B,S16,0.90,0.70,0.6300,0.6300
8,Betis - Valladolid,B-C,D12,0.75,0.45,0.3375,0.3375
9,Atlético - Getafe,B-B,S18,0.90,0.80,0.7200,0.7200


La audiencia total esperada es: 6.856000000000001 M. 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,D20,1.30,1.00,1.3000,1.3000
1,Valencia - R.Sociedad,B-A,D18,1.30,0.85,1.1050,1.1050
2,Mallorca - Eibar,C-C,L20,0.47,0.40,0.1880,0.1880
3,Athletic - Barcelona,B-A,S20,1.30,1.00,1.3000,1.3000
4,Leganés - Osasuna,C-C,V20,0.47,0.40,0.1880,0.1880
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,D16,0.90,0.75,0.6750,0.6750
7,Espayol - Sevilla,B-B,S18,0.90,0.80,0.7200,0.7200
8,Betis - Valladolid,B-C,D12,0.75,0.45,0.3375,0.3375
9,Atlético - Getafe,B-B,S16,0.90,0.70,0.6300,0.6300


La audiencia total esperada es: 6.856000000000001 M. 



**(*)Calcula la complejidad del algoritmo**

**Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios**


Aquí tengo duda, es decir lo que debe ser aleatorio ¿es la jornada a analizar? Me refiero a una combinatoria de partidos iniciales diferentes.

Mi duda viene porque no sé si se refiere a eso, o a que calcule la máxima audiencia que se produce entre todas las jornadas posibles en la mejor combinación de horarios para cada una.

Si es la primera, aquí dejo una función primitiva que será la base del juego de entradas aleatorios que presentaré más adelante, es decir, mi aleatoriedad vendrá por el enfrentamiento que se produzca en cada partido, manteniendo las categorías, ponderaciones y la correción por coincidencia. Es lo que yo le encuentro sentido, aplicar el problema en una jornada con enfrentamientos aleatorios para determinar cual es el horario óptimo para esa jornada.

Problema 2. Organizar los horarios de partidos de La Liga (Jornada aleatoria)

In [ ]:
# Añadir Horarios - Versión Beta (Aleatorios sin restrincciones) debe tenerlas

import random
import numpy as np
import pandas as pd

dict_equipos = {'Celta' : 'B', 'Real Madrid' : 'A', 'Valencia' : 'B', 'R.Sociedad' : 'A', 'Mallorca' : 'C', 'Eibar' : 'C',
           'Athletic' : 'B', 'Barcelona' : 'A', 'Leganés' : 'C', 'Osasuna' : 'C', 'Villareal' : 'B', 'Granada' : 'C',
           'Alavés' : 'B', 'Levante' : 'B', 'Espayol' : 'B', 'Sevilla' : 'B', 'Betis' : 'B', 'Valladolid' : 'C', 'Atlético' : 'B',
           'Getafe' : 'B'
           }

def clasificar_partido(cat_1, cat_2):

  # Generamos la tabla de audiencia base
  audiencia_base = {
      'A' : {'A' : 2},
      'B' : {'A' : 1.3, 'B' : 0.9},
      'C' : {'A' : 1, 'B' : 0.75, 'C' : 0.47}
  }

  return audiencia_base.get(cat_1,{}).get(cat_2,audiencia_base.get(cat_2,{}).get(cat_1))

def Correcion_por_coincidencia(lista_horario_partido): # devuelve una lista con las diferentes correciones en base a la coincidencia
  Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}
  lista_correciones = []

  for hora in lista_horario_partido:
    contador = lista_horario_partido.count(hora) - 1
    lista_correciones.append(Coincidencias['-%'][contador])
  return lista_correciones

def generar_partidos (dict_equipos):
  "Función que devuelve los partidos que se van a jugar"
  lista_partidos = [] # lista que guarda los distintos partidos de una jornada
  lista_categorias = [] # lista que guarda las distintas categorias de los partidos de una jornada
  lista_bases = [] # lista con las bases de audiencia de cada partido en funcion de la categoría
  lista_equipos = list(dict_equipos.keys()) #lista con todos los equipos
  lista_equipos_disponibles = lista_equipos[:]

  # Voy a ir llevando un diccionario en el que guarde los partidos jugados
  for i in range (0, len(lista_equipos),2):
    equipo_1 = random.choice(lista_equipos_disponibles)
    categoria_equipo_1 = dict_equipos[str(equipo_1)]
    lista_equipos_disponibles.remove(equipo_1)
    equipo_2 = random.choice(lista_equipos_disponibles)
    categoria_equipo_2 = dict_equipos[str(equipo_2)]
    lista_equipos_disponibles.remove(equipo_2)
    partido = str(equipo_1) + ' - ' + str(equipo_2)
    categoria = categoria_equipo_1 + '-' + categoria_equipo_2
    base = clasificar_partido(categoria_equipo_1,categoria_equipo_2)
    lista_partidos.append(partido)
    lista_categorias.append(categoria)
    lista_bases.append(base)

  Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                 'D20' : 1, 'L20' : 0.4}
  listado_horarios = list(Ponderacion.keys())
  lista_horario_partido = [random.choice(listado_horarios) for partido in lista_partidos]
  lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]

  return lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones

lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones = generar_partidos(dict_equipos)
lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)

dict_jornada = {
    'Partido' : lista_partidos,
    'Categorías' : lista_categorias,
    'Horario' : lista_horario_partido,
    'Base(Mill.)' : lista_bases,
    'Ponderación' : lista_ponderaciones,
    'Base*Ponderación' : lista_base_ponderacion,
    'Correción Coincidencia' : lista_correciones_corregida,
}

df_jornada = pd.DataFrame(dict_jornada)
display(df_jornada)
Total = np.sum(lista_correciones_corregida)
print('\n Total: ',Total)

,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Mallorca - Sevilla,C-B,D18,0.75,0.85,0.6375,0.478125
1,Real Madrid - Villareal,A-B,V20,1.30,0.40,0.5200,0.520000
2,Getafe - Levante,B-B,D16,0.90,0.75,0.6750,0.371250
3,Betis - Celta,B-B,S18,0.90,0.80,0.7200,0.720000
4,Atlético - Osasuna,B-C,D16,0.75,0.75,0.5625,0.309375
5,Alavés - Eibar,B-C,D16,0.75,0.75,0.5625,0.309375
6,Valencia - Leganés,B-C,S12,0.75,0.55,0.4125,0.412500
7,Granada - R.Sociedad,C-A,L20,1.00,0.40,0.4000,0.400000
8,Espayol - Athletic,B-B,S20,0.90,1.00,0.9000,0.900000
9,Barcelona - Valladolid,A-C,D18,1.00,0.85,0.8500,0.637500



 Total:  5.058125


**Aplica el algoritmo al juego de datos generado**

In [ ]:
# Resolver problema de jornada aleatoria con backtraking y poda

**Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo**

**Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño**

Aumentando número de partidos (influye más) u horarios y calcular el máximo de máximos, es decir, dar el horario que da máx audiencia en la jornada que da la máx audiencia posible (implicaría resolver este problema para todas las jornadas posibles en un algoritmo de fuerza bruta, que se podría reducir mucho aplicando una poda en base a la audiencia máxima posible a obtenerse.